In [1]:
import polars as pl
import requests
import zipfile
from io import BytesIO, TextIOWrapper
from datetime import datetime, date
import warnings
from urllib3.exceptions import InsecureRequestWarning
import os
import tempfile
import psycopg2
from psycopg2.extras import execute_values

# Suppress insecure request warnings
warnings.simplefilter('ignore', InsecureRequestWarning)

In [11]:
# Configurações de banco de dados
DATABASE_CONFIG = {
    "host": "localhost",
    "database": "meu_banco",
    "user": "admin",
    "password": "admin_password",
    "port": 5432
}

# --- Definições de colunas, mercados, etc. ---
FIELD_SIZES = {
    'TIPO_DE_REGISTRO': 2, 'DATA_DO_PREGAO': 8, 'CODIGO_BDI': 2, 'CODIGO_DE_NEGOCIACAO': 12,
    'TIPO_DE_MERCADO': 3, 'NOME_DA_EMPRESA': 12, 'ESPECIFICACAO_DO_PAPEL': 10,
    'PRAZO_EM_DIAS_DO_MERCADO_A_TERMO': 3, 'MOEDA_DE_REFERENCIA': 4, 'PRECO_DE_ABERTURA': 13,
    'PRECO_MAXIMO': 13, 'PRECO_MINIMO': 13, 'PRECO_MEDIO': 13, 'PRECO_ULTIMO_NEGOCIO': 13,
    'PRECO_MELHOR_OFERTA_DE_COMPRA': 13, 'PRECO_MELHOR_OFERTA_DE_VENDAS': 13,
    'NUMERO_DE_NEGOCIOS': 5, 'QUANTIDADE_NEGOCIADA': 18, 'VOLUME_TOTAL_NEGOCIADO': 18,
    'PRECO_DE_EXERCICIO': 13, 'INDICADOR_DE_CORRECAO_DE_PRECOS': 1, 'DATA_DE_VENCIMENTO': 8,
    'FATOR_DE_COTACAO': 7, 'PRECO_DE_EXERCICIO_EM_PONTOS': 13, 'CODIGO_ISIN': 12,
    'NUMERO_DE_DISTRIBUICAO': 3
}

FLOAT_COLUMNS = ['PRECO_DE_ABERTURA', 'PRECO_MAXIMO', 'PRECO_MINIMO', 'PRECO_MEDIO',
                'PRECO_ULTIMO_NEGOCIO', 'PRECO_MELHOR_OFERTA_DE_COMPRA',
                'PRECO_MELHOR_OFERTA_DE_VENDAS', 'PRECO_DE_EXERCICIO',
                'PRECO_DE_EXERCICIO_EM_PONTOS', 'VOLUME_TOTAL_NEGOCIADO', 'QUANTIDADE_NEGOCIADA']

DATE_COLUMNS = ["DATA_DO_PREGAO", "DATA_DE_VENCIMENTO"]

CODBDI = {
    '02': "LOTE_PADRAO"
    # Simplificado para o que realmente usamos no filtro
}

BASE_URL = "https://bvmf.bmfbovespa.com.br/InstDados/SerHist/COTAHIST"

In [3]:
# Função para inserir dados no banco
def insert_into_database(df, batch_size=10000):
    """Insere os dados processados no banco de dados usando psycopg2 com execute_values."""
    if df.is_empty():
        print("DataFrame vazio, nada a inserir.")
        return 0

    conn = None
    inserted = 0
    try:
        conn = psycopg2.connect(**DATABASE_CONFIG)
        with conn.cursor() as cursor:
            # Preparar dados para inserção
            data = df.to_pandas().to_dict('records')

            # Inserir em lotes para melhor performance
            for i in range(0, len(data), batch_size):
                batch = data[i:i+batch_size]
                values = [(
                    row['codigo_isin'],
                    row['data_pregao'],
                    row['abertura'],
                    row['fechamento'],
                    row['numero_de_negocios'],
                    row['quantidade_negociada'],
                    row['volume_negociado']
                ) for row in batch]

                # Usar execute_values para inserção em lote
                execute_values(
                    cursor,
                    """
                    INSERT INTO cotacao
                    (codigo_isin, data_pregao, abertura, fechamento, numero_de_negocios,
                     quantidade_negociada, volume_negociado)
                    VALUES %s
                    ON CONFLICT (codigo_isin, data_pregao) DO NOTHING
                    """,
                    values
                )

            conn.commit()
            inserted = len(data)
            print(f"Inseridos/Ignorados ~{inserted} registros na tabela cotacao.")

    except Exception as e:
        if conn:
            conn.rollback()
        print(f"Erro ao inserir no banco de dados: {e}")
    finally:
        if conn:
            conn.close()

    return inserted

In [4]:
# Função para obter tickers existentes no banco
def get_existing_tickers():
    """Busca os tickers existentes no banco de dados."""
    conn = None
    try:
        conn = psycopg2.connect(**DATABASE_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute("SELECT codigo_isin FROM ticker")
            return set(row[0] for row in cursor.fetchall())
    except Exception as e:
        print(f"Erro ao buscar tickers: {e}")
        return set()
    finally:
        if conn:
            conn.close()

In [5]:

# Função para processar o arquivo de cotações
def process_file(file_path, existing_tickers):
    """Processa o arquivo de cotações usando Polars."""
    print(f"Processando arquivo: {file_path}")

    # Definir esquema para leitura mais eficiente
    schema = {
        'TIPO_DE_REGISTRO': pl.Utf8,
        'DATA_DO_PREGAO': pl.Utf8,
        'CODIGO_BDI': pl.Utf8,
        'CODIGO_ISIN': pl.Utf8,
        'PRECO_DE_ABERTURA': pl.Utf8,
        'PRECO_ULTIMO_NEGOCIO': pl.Utf8,
        'NUMERO_DE_NEGOCIOS': pl.Utf8,
        'QUANTIDADE_NEGOCIADA': pl.Utf8,
        'VOLUME_TOTAL_NEGOCIADO': pl.Utf8
    }

    # Criar uma lista de posições de coluna baseada no FIELD_SIZES
    column_positions = []
    current_pos = 0
    for col, width in FIELD_SIZES.items():
        column_positions.append((current_pos, current_pos + width))
        current_pos += width

    # Filtrar apenas as colunas que precisamos
    needed_columns = ['TIPO_DE_REGISTRO', 'DATA_DO_PREGAO', 'CODIGO_BDI', 'PRECO_DE_ABERTURA',
                      'PRECO_ULTIMO_NEGOCIO', 'NUMERO_DE_NEGOCIOS', 'QUANTIDADE_NEGOCIADA',
                      'VOLUME_TOTAL_NEGOCIADO', 'CODIGO_ISIN']

    needed_positions = []
    column_names = []
    for i, col in enumerate(FIELD_SIZES.keys()):
        if col in needed_columns:
            needed_positions.append(column_positions[i])
            column_names.append(col)

    # Ler o arquivo usando Polars com leitura por colunas fixas
    # Pular a primeira linha (cabeçalho) e usar LazyFrame para processamento eficiente
    df = pl.read_csv(
        file_path,
        has_header=False,
        skip_rows=1,
        encoding='latin1',
        separator='\n',  # Cada linha é um registro
        columns=[0],     # Lê apenas a primeira coluna que contém toda a linha
        new_columns=["line"]  # Renomeia para "line"
    ).lazy()

    # Extrair as colunas necessárias da linha usando expressões
    for i, (col_name, (start, end)) in enumerate(zip(column_names, needed_positions)):
        df = df.with_columns([
            pl.col("line").str.slice(start, end - start).alias(col_name)
        ])

    # Filtrar linhas que não são trailer (TIPO_DE_REGISTRO != '99')
    df = df.filter(pl.col("TIPO_DE_REGISTRO") != "99")

    # Filtrar por CODIGO_BDI == "02" (LOTE_PADRAO)
    df = df.filter(pl.col("CODIGO_BDI") == "02")

    # Converter tipos de dados
    df = df.with_columns([
        # Converter datas
        pl.col("DATA_DO_PREGAO").str.to_date("%Y%m%d").alias("data_pregao"),

        # Converter valores numéricos
        (pl.col("PRECO_DE_ABERTURA").cast(pl.Float64) / 100).alias("abertura"),
        (pl.col("PRECO_ULTIMO_NEGOCIO").cast(pl.Float64) / 100).alias("fechamento"),
        pl.col("NUMERO_DE_NEGOCIOS").cast(pl.Int64).alias("numero_de_negocios"),
        pl.col("QUANTIDADE_NEGOCIADA").cast(pl.Float64).alias("quantidade_negociada"),
        pl.col("VOLUME_TOTAL_NEGOCIADO").cast(pl.Float64).alias("volume_negociado"),

        # Manter CODIGO_ISIN como está
        pl.col("CODIGO_ISIN").alias("codigo_isin")
    ])

    # Selecionar apenas as colunas finais e filtrar por tickers existentes
    df = df.select([
        "codigo_isin", "data_pregao", "abertura", "fechamento",
        "numero_de_negocios", "quantidade_negociada", "volume_negociado"
    ]).filter(
        pl.col("codigo_isin").is_in(list(existing_tickers))
    )

    # Executar o processamento e retornar o DataFrame materializado
    return df.collect()

In [13]:
def download_and_extract(url_suffix):
    """Baixa e extrai o arquivo ZIP da B3."""
    url = f"{BASE_URL}{url_suffix}"
    tmp_zip_path = None
    tmp_txt_path = None

    try:
        print(f"Baixando {url}...")
        response = requests.get(url, verify=False, stream=True)
        response.raise_for_status()

        # Salvar ZIP em arquivo temporário
        with tempfile.NamedTemporaryFile(delete=False, suffix=".zip") as tmp_zip_file:
            for chunk in response.iter_content(chunk_size=8192 * 16):
                tmp_zip_file.write(chunk)
            tmp_zip_path = tmp_zip_file.name

        print(f"Arquivo ZIP salvo em: {tmp_zip_path}")

        # Criar um arquivo temporário para o TXT
        with tempfile.NamedTemporaryFile(delete=False, suffix=".txt") as tmp_txt_file:
            tmp_txt_path = tmp_txt_file.name

        # Fechar e remover o arquivo temporário vazio para evitar conflitos
        os.remove(tmp_txt_path)

        # Extrair TXT do ZIP
        file_name_in_zip = url_suffix.replace('.ZIP', '.TXT')
        file_name_in_zip = "COTAHIST" + file_name_in_zip

        print(f"Procurando arquivo {file_name_in_zip} no ZIP...")

        with zipfile.ZipFile(tmp_zip_path, 'r') as zf:
            if file_name_in_zip not in zf.namelist():
                print(f"ERRO: Arquivo {file_name_in_zip} não encontrado no ZIP")
                print(f"Arquivos no ZIP: {zf.namelist()}")
                return None

            print(f"Extraindo {file_name_in_zip} para {tmp_txt_path}...")

            # Extrair diretamente para o caminho temporário
            with zf.open(file_name_in_zip) as source, open(tmp_txt_path, 'wb') as target:
                target.write(source.read())

        print(f"Arquivo extraído com sucesso para: {tmp_txt_path}")
        return tmp_txt_path

    except Exception as e:
        print(f"Erro ao baixar/extrair arquivo {url}: {e}")
        import traceback
        traceback.print_exc()
        return None
    finally:
        # Limpar arquivo ZIP temporário
        if tmp_zip_path and os.path.exists(tmp_zip_path):
            os.remove(tmp_zip_path)
            print(f"Arquivo ZIP temporário removido: {tmp_zip_path}")

In [7]:
# Funções para gerar sufixos de arquivos
def get_years_range():
    current_year = datetime.now().year
    return range(2024, current_year + 1)

def get_months_current_year():
    current_date = datetime.now()
    current_month = current_date.month
    current_year = current_date.year
    return [f"_M{month:02d}{current_year}.ZIP" for month in range(1, current_month)]

def get_days_current_month():
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    current_day = current_date.day
    if current_day <= 1:
        return []
    return [f"_D{day:02d}{current_month:02d}{current_year}.ZIP" for day in range(1, current_day)]

In [14]:
# Função principal de ETL
def run_etl():
    """Função principal que executa o ETL completo."""
    print("Iniciando ETL de cotações B3...")

    # Obter tickers existentes
    existing_tickers = get_existing_tickers()
    print(f"Encontrados {len(existing_tickers)} tickers no banco de dados.")

    if not existing_tickers:
        print("AVISO: Nenhum ticker encontrado. Nenhuma cotação será inserida.")
        return

    # Processar arquivos anuais
    print("\n=== Processando arquivos anuais ===")
    for year in get_years_range():
        url_suffix = f"_A{year}.ZIP"
        print(f"Processando ano {year}...")

        # Download e extração
        txt_path = download_and_extract(url_suffix)
        if not txt_path:
            print(f"Falha ao baixar/extrair arquivo do ano {year}")
            continue

        try:
            # Processar arquivo
            df = process_file(txt_path, existing_tickers)

            # Inserir no banco
            if not df.is_empty():
                inserted = insert_into_database(df)
                print(f"Processamento do ano {year} concluído. Registros: {inserted}")
            else:
                print(f"Nenhum registro válido encontrado para o ano {year}")
        finally:
            # Limpar arquivo temporário
            if os.path.exists(txt_path):
                os.remove(txt_path)

    # Processar arquivos mensais
    print("\n=== Processando arquivos mensais ===")
    for month_suffix in get_months_current_year():
        print(f"Processando mês {month_suffix}...")

        # Download e extração
        txt_path = download_and_extract(month_suffix)
        if not txt_path:
            print(f"Falha ao baixar/extrair arquivo do mês {month_suffix}")
            continue

        try:
            # Processar arquivo
            df = process_file(txt_path, existing_tickers)

            # Inserir no banco
            if not df.is_empty():
                inserted = insert_into_database(df)
                print(f"Processamento do mês {month_suffix} concluído. Registros: {inserted}")
            else:
                print(f"Nenhum registro válido encontrado para o mês {month_suffix}")
        finally:
            # Limpar arquivo temporário
            if os.path.exists(txt_path):
                os.remove(txt_path)

    # Processar arquivos diários
    print("\n=== Processando arquivos diários ===")
    for day_suffix in get_days_current_month():
        print(f"Processando dia {day_suffix}...")

        # Download e extração
        txt_path = download_and_extract(day_suffix)
        if not txt_path:
            print(f"Falha ao baixar/extrair arquivo do dia {day_suffix}")
            continue

        try:
            # Processar arquivo
            df = process_file(txt_path, existing_tickers)

            # Inserir no banco
            if not df.is_empty():
                inserted = insert_into_database(df)
                print(f"Processamento do dia {day_suffix} concluído. Registros: {inserted}")
            else:
                print(f"Nenhum registro válido encontrado para o dia {day_suffix}")
        finally:
            # Limpar arquivo temporário
            if os.path.exists(txt_path):
                os.remove(txt_path)

    print("\nETL de cotações B3 concluído com sucesso!")


In [ ]:
# Executar o ETL
if __name__ == "__main__":
    run_etl()

Iniciando ETL de cotações B3...
Encontrados 520 tickers no banco de dados.

=== Processando arquivos anuais ===
Processando ano 2024...
Baixando https://bvmf.bmfbovespa.com.br/InstDados/SerHist/COTAHIST_A2024.ZIP...
Arquivo ZIP salvo em: C:\Users\felip\AppData\Local\Temp\tmpqp7ku3lf.zip
Procurando arquivo COTAHIST_A2024.TXT no ZIP...
Extraindo COTAHIST_A2024.TXT para C:\Users\felip\AppData\Local\Temp\tmpmf8ck2q6.txt...
Arquivo extraído com sucesso para: C:\Users\felip\AppData\Local\Temp\tmpmf8ck2q6.txt
Arquivo ZIP temporário removido: C:\Users\felip\AppData\Local\Temp\tmpqp7ku3lf.zip
Processando arquivo: C:\Users\felip\AppData\Local\Temp\tmpmf8ck2q6.txt
Inseridos/Ignorados ~78057 registros na tabela cotacao.
Processamento do ano 2024 concluído. Registros: 78057
Processando ano 2025...
Baixando https://bvmf.bmfbovespa.com.br/InstDados/SerHist/COTAHIST_A2025.ZIP...
Arquivo ZIP salvo em: C:\Users\felip\AppData\Local\Temp\tmpr1fwp4qp.zip
Procurando arquivo COTAHIST_A2025.TXT no ZIP...
Extr